In [9]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [10]:
cc = CasosConfirmados()
# cc.read()
# cc.save()
cc.load()
notifica = Notifica()
# notifica.download_todas_notificacoes()
# notifica.save()
notifica.load()
print(cc.shape())
print(notifica.shape())

(569911, 10388, 420248, 139275)
(2238828, 1028707, 10529, 1197502, 2090)


In [11]:
casos_confirmados = cc.get_casos()
casos_notifica = notifica.get_casos()
print(casos_notifica.shape)

(1849633, 54)


In [12]:
merge_casos_confirmados = pd.merge(casos_confirmados,casos_notifica[['id','hash_diag']],how='left',on='hash_diag')
print(merge_casos_confirmados.shape)

(569911, 24)


In [13]:
merge_casos_confirmados.loc[merge_casos_confirmados['id'].notnull()]

,ordem,identificacao,ibge_resid,ibge_atend,paciente,sexo,idade,mun_resid,mun_atend,rs,laboratorio,data_diagnostico,data_comunicacao,data_1o_sintomas,obito,data_obito,data_com_obito,recuperado,ativo,excluir,hash_resid,hash_atend,hash_diag,id
48,49,49,4119152,4115606,ADRIANO RODRIGUES DE ALMEIDA,M,37,PINHAIS,PINHAIS,2,LACEN,2020-03-21,2020-03-22,2020-03-15,0,NaT,NaT,1,0,NaN,ADRIANORODRIGUESDEALMEIDA37PINHAIS,ADRIANORODRIGUESDEALMEIDA37PINHAIS,ADRIANORODRIGUESDEALMEIDA21032020,104129.0
58,59,59,4104808,4101705,DEBORA ANDREIA ALVES MOREIRA,F,42,CASCAVEL,CASCAVEL,10,DASA,2020-03-22,2020-03-23,2020-03-13,0,NaT,NaT,1,0,NaN,DEBORAANDREIAALVESMOREIRA42CASCAVEL,DEBORAANDREIAALVESMOREIRA42CASCAVEL,DEBORAANDREIAALVESMOREIRA22032020,75221.0
61,62,62,4106902,4118204,BEATRIZ GRABOSKI SILVA,F,61,CURITIBA,CURITIBA,2,LACEN,2020-03-23,2020-03-24,2020-03-16,0,NaT,NaT,1,0,NaN,BEATRIZGRABOSKISILVA61CURITIBA,BEATRIZGRABOSKISILVA61CURITIBA,BEATRIZGRABOSKISILVA23032020,113404.0
67,68,68,4104808,4125506,BIANCA PEPES DE OLIVEIRA PERLIN,F,23,CASCAVEL,CASCAVEL,10,LACEN,2020-03-23,2020-03-24,2020-03-14,0,NaT,NaT,1,0,NaN,BIANCAPEPESDEOLIVEIRAPERLIN23CASCAVEL,BIANCAPEPESDEOLIVEIRAPERLIN23CASCAVEL,BIANCAPEPESDEOLIVEIRAPERLIN23032020,75937.0
81,82,82,4106902,4106902,RENAN BARBOZA DA SILVA,M,32,CURITIBA,CURITIBA,2,LACEN,2020-03-25,2020-03-25,2020-03-14,0,NaT,NaT,1,0,NaN,RENANBARBOZADASILVA32CURITIBA,RENANBARBOZADASILVA32CURITIBA,RENANBARBOZADASILVA25032020,112368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569906,569907,651561,4106209,4106209,ZENI BENEDITA DOS SANTOS,F,58,CONTENDA,CONTENDA,2,COVID-19 BIOLOGIA MOLECULAR,2021-02-07,2021-02-07,2021-02-03,0,NaT,NaT,0,1,NaN,ZENIBENEDITADOSSANTOS58CONTENDA,ZENIBENEDITADOSSANTOS58CONTENDA,ZENIBENEDITADOSSANTOS07022021,2292266.0
569907,569908,651562,4104204,4104204,ZENI DOS SANTOS,F,49,CAMPO LARGO,CAMPO LARGO,2,COVID-19 BIOLOGIA MOLECULAR,2021-02-06,2021-02-07,2021-02-02,0,NaT,NaT,0,1,NaN,ZENIDOSSANTOS49CAMPOLARGO,ZENIDOSSANTOS49CAMPOLARGO,ZENIDOSSANTOS06022021,2295624.0
569908,569909,651563,4112108,4112108,ZEQUIEL GOMES PEREIRA,M,54,JANDAIA DO SUL,JANDAIA DO SUL,16,COVID-19 BIOLOGIA MOLECULAR,2021-02-06,2021-02-07,2021-02-01,0,NaT,NaT,0,1,NaN,ZEQUIELGOMESPEREIRA54JANDAIADOSUL,ZEQUIELGOMESPEREIRA54JANDAIADOSUL,ZEQUIELGOMESPEREIRA06022021,2269607.0
569909,569910,651564,4115200,4115200,ZEZILDA DE ARRUDA LIMA,F,64,MARINGA,MARINGA,15,TESTE MOLECULAR (RT-PCR),2021-02-06,2021-02-07,2021-02-02,0,NaT,NaT,0,1,NaN,ZEZILDADEARRUDALIMA64MARINGA,ZEZILDADEARRUDALIMA64MARINGA,ZEZILDADEARRUDALIMA06022021,2304191.0


In [14]:
merge_casos_confirmados.loc[merge_casos_confirmados['id'].isnull()]

,ordem,identificacao,ibge_resid,ibge_atend,paciente,sexo,idade,mun_resid,mun_atend,rs,laboratorio,data_diagnostico,data_comunicacao,data_1o_sintomas,obito,data_obito,data_com_obito,recuperado,ativo,excluir,hash_resid,hash_atend,hash_diag,id
0,1,1,4105508,4108304,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,13,LACEN,2020-03-12,2020-03-12,NaT,0,NaT,NaT,1,0,NaN,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN
1,2,2,4106902,4117107,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,2,LACEN,2020-03-12,2020-03-12,2020-03-07,0,NaT,NaT,1,0,NaN,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN
2,3,3,4106902,4127700,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,2,LACEN,2020-03-12,2020-03-12,2020-03-08,0,NaT,NaT,1,0,NaN,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN
3,4,4,4106902,4112009,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,2,GENOPRIMER,2020-03-16,2020-03-16,2020-03-11,0,NaT,NaT,1,0,NaN,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE16032020,NaN
4,5,5,4106902,4125506,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,2,GENOPRIMER,2020-03-18,2020-03-18,2020-03-15,0,NaT,NaT,1,0,NaN,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN18032020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569354,569355,651009,4115200,4115200,MARIA DE FATIMA MARTINS AGOSTINHO BARBOSA,F,58,MARINGA,MARINGA,15,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-02-04,2021-02-07,2021-02-01,0,NaT,NaT,0,1,NaN,MARIADEFATIMAMARTINSAGOSTINHOBARBOSA58MARINGA,MARIADEFATIMAMARTINSAGOSTINHOBARBOSA58MARINGA,MARIADEFATIMAMARTINSAGOSTINHOBARBOSA04022021,NaN
569559,569560,651214,4109609,4109609,PAULO MAURICIO FLORA,M,29,GUARATUBA,GUARATUBA,1,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-02-06,2021-02-07,2021-02-01,0,NaT,NaT,0,1,NaN,PAULOMAURICIOFLORA29GUARATUBA,PAULOMAURICIOFLORA29GUARATUBA,PAULOMAURICIOFLORA06022021,NaN
569729,569730,651384,4108403,4108403,SIMONE GUADANHIN SEVERINO,F,34,FRANCISCO BELTRAO,FRANCISCO BELTRAO,8,COVID-19 BIOLOGIA MOLECULAR,2021-02-05,2021-02-07,2021-01-27,0,NaT,NaT,0,1,NaN,SIMONEGUADANHINSEVERINO34FRANCISCOBELTRAO,SIMONEGUADANHINSEVERINO34FRANCISCOBELTRAO,SIMONEGUADANHINSEVERINO05022021,NaN
569776,569777,651431,4114609,4114609,TEREZINHA ALVES DE MACENI,F,42,MARECHAL CANDIDO RONDON,MARECHAL CANDIDO RONDON,20,COVID-19 BIOLOGIA MOLECULAR,2021-02-06,2021-02-07,2021-01-31,0,NaT,NaT,0,1,NaN,TEREZINHAALVESDEMACENI42MARECHALCANDIDORONDON,TEREZINHAALVESDEMACENI42MARECHALCANDIDORONDON,TEREZINHAALVESDEMACENI06022021,NaN
